In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/usr0/home/naveenr/projects/food_rescue_feedback')

In [3]:
import numpy as np
import random 
import matplotlib.pyplot as plt
from feedback.fr_feedback import *
from feedback.generate_report import *
import feedback.secret as secret
from feedback.database import open_connection
import pandas as pd

In [4]:
start_date = "2024-05-20"
end_date = "2024-05-26"

## Donors

In [5]:
db_name = "fr_new" 
username = secret.database_username 
password = secret.database_password 
ip_address = "localhost"
port = secret.database_port

In [6]:
connection_dict = open_connection(db_name,username,password,ip_address,port)
connection = connection_dict['connection']
cursor = connection_dict['cursor']

In [7]:
# feedbacks = get_feedback_by_date(connection,start_date,end_date)
# feedbacks.to_csv('../../results/reports/feedbacks_{}_{}.csv'.format(start_date,end_date), index=False)

In [8]:
# annotated_feedback = generate_prompts_and_analyze_feedback(feedbacks)
# annotated_feedback.to_csv('../../results/reports/labeled_feedbacks_{}_{}.csv'.format(start_date,end_date), index=False)

In [9]:
feedbacks = pd.read_csv('../../results/reports/feedbacks_{}_{}.csv'.format(start_date,end_date))
annotated_feedback = pd.read_csv('../../results/reports/labeled_feedbacks_{}_{}.csv'.format(start_date,end_date))

In [13]:
feedbacks_labeled = pd.read_csv('../../results/annotations/labeled_data.csv')
feedbacks_labeled = feedbacks_labeled[:10]
feedbacks_labeled

,id,donation_id,recipient_location_id,published_at,created_at,updated_at,state,user_id,rescue_group_id,distance,...,recipient_name,volunteer_comment,inadequate_food_jingwu,donor_contact_problem_jingwu,recipient_problem_jingwu,info_update_jingwu,inadequate_food_naveen,donor_contact_problem_naveen,recipient_problem_naveen,info_update_naveen
0,411243,332242,261,00:00.0,19:19.1,46:18.9,8,26039,NaN,1.357075,...,Salvation Army,All the people I interacted with were extremel...,1,1,0,1,1,0.0,1.0,1.0
1,394294,332582,5852,00:32.0,24:58.8,52:03.4,8,37203,NaN,2.565020,...,Margaret B Rost School,Donation was only one very small bag of food a...,1,0,1,0,1,0.0,1.0,0.0
2,400184,333430,8634,50:53.2,17:29.7,39:39.4,8,44991,NaN,11.203944,...,Poly High School,Ejiro is no longer a contact and should be rem...,0,0,1,1,0,0.0,1.0,1.0
3,410305,331332,2674,00:00.0,57:19.7,23:27.3,8,11822,NaN,3.306937,...,Free Farm Stand,Alex & farmstand are no longer at this locatio...,0,0,1,1,0,0.0,1.0,1.0
4,360938,331777,364,01:30.6,31:59.4,51:27.0,8,45090,NaN,0.577875,...,Islamic Center of Pittsburgh Food Pantry,No one was at the Islamic Center and no one an...,0,0,1,0,0,0.0,1.0,0.0
5,404598,334958,357,00:00.0,25:19.5,13:49.3,8,14046,NaN,2.310258,...,Wilkinsburg Community Ministry,Scolded for not coming in AM ‘as usual’… & wro...,0,0,1,0,0,0.0,1.0,0.0
6,400179,333431,8638,51:02.0,13:55.5,39:16.9,8,18430,NaN,3.041612,...,Millikan High School,"The school personnel , Aimee Uresti contacted ...",1,0,1,0,0,0.0,1.0,0.0
7,404770,331579,8298,30:30.7,44:47.6,27:57.9,8,47820,NaN,0.689722,...,Hamden Fire Department,"Fire dept got a call, no one available so left...",0,0,1,0,0,0.0,1.0,0.0
8,402687,333413,1709,46:11.1,22:47.7,29:13.5,8,44994,NaN,8.676827,...,Precious Life Shelter Inc,I called the person from the Food Finders app ...,1,1,0,0,1,1.0,0.0,0.0
9,387448,333166,1284,30:30.9,27:15.5,12:24.6,8,14022,NaN,2.558185,...,Concordia Lutheran Church,This pick up is not timely. They dont process ...,1,1,0,0,1,1.0,0.0,0.0


In [16]:
labeled_prompts = generate_prompts_and_analyze_feedback(feedbacks_labeled)

On feedback 1 out of 10
On feedback 2 out of 10
On feedback 3 out of 10
Error processing feedback 2 for task info_update: 'info_update'
On feedback 4 out of 10
On feedback 5 out of 10
On feedback 6 out of 10
On feedback 7 out of 10
On feedback 8 out of 10
Error processing feedback 7 for task info_update: 'info_update'
On feedback 9 out of 10
On feedback 10 out of 10


In [17]:
labeled_prompts.to_csv('../../results/annotations/labeled_data_with_llm_gpt_4.csv')

In [17]:
# Calculate statistics and generate the report
donor_statistics = calculate_donor_statistics(feedbacks, annotated_feedback)
recipient_statistics = calculate_recipient_statistics(feedbacks, annotated_feedback)
report_info = info_update(annotated_feedback)

# Save the report to a CSV file
donor_statistics.to_csv('../../results/reports/report_donor_{}_{}.csv'.format(start_date,end_date), index=False)
recipient_statistics.to_csv('../../results/reports/report_recipient_{}_{}.csv'.format(start_date,end_date), index=False)
report_info.to_csv('../../results/reports/report_info_{}_{}.csv'.format(start_date,end_date), index=False)